# Imports

In [1]:
import findspark

findspark.init('C:/spark')

In [36]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import length
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer, VectorAssembler
from pyspark.ml.classification import NaiveBayes
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Create a session

In [3]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

# Read data

In [9]:
data = spark.read.csv('../../data/smsspamcollection/SMSSpamCollection', sep='\t')

In [10]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [11]:
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

In [12]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [14]:
data = data.withColumn('length', length(data['text']))

In [15]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [16]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [19]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

In [21]:
clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')

In [23]:
nb = NaiveBayes()

In [25]:
preprocessing_pipe = Pipeline(stages=[
    ham_spam_to_numeric,
    tokenizer,
    stop_remove,
    count_vec,
    idf,
    clean_up
])

In [26]:
cleaner = preprocessing_pipe.fit(data)
clean_data = cleaner.transform(data)

In [29]:
clean_data = clean_data.select('label', 'features')

In [30]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [31]:
training, test = clean_data.randomSplit([0.7, 0.3])

In [32]:
spam_detector = nb.fit(training)

In [33]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [34]:
test_results = spam_detector.transform(test)

In [35]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-789.55585174179...|[1.0,3.9570049479...|       0.0|
|  0.0|(13424,[0,1,2,41,...|[-1087.8793004638...|[0.99999999999999...|       0.0|
|  0.0|(13424,[0,1,3,9,1...|[-570.92352390183...|[1.0,5.1957656888...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-882.83029341175...|[1.0,3.5065471515...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[-658.94696638459...|[1.0,5.6912047393...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-557.26106745002...|[1.0,1.6881550147...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-557.26106745002...|[1.0,1.6881550147...|       0.0|
|  0.0|(13424,[0,1,12,33...|[-442.19433371599...|[1.0,7.4017753980...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-1359.1691136577...|[1.0,9.5408027921...|       0.0|
|  0.0|(13424,[0

In [41]:
acc_eval = MulticlassClassificationEvaluator()

In [42]:
acc = acc_eval.evaluate(test_results)

In [43]:
print(f'ACC of Naive Bayes Model: {acc}')

ACC of Naive Bayes Model: 0.9226603101328478
